# Tiling Test - Updated with New Features

This notebook demonstrates the improved AbFab.py features:
1. **Dual filter options**: Gaussian (default) vs von Kármán
2. **Spreading rate utilities**: Auto-calculate parameters from spreading rate
3. **Comparison**: Side-by-side results with different settings

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import xarray as xr
import time
import pygmt
import AbFab as af

%load_ext autoreload
%autoreload 2

%matplotlib inline

## Load Data (Same as Original)

In [ ]:
print("Loading seafloor age and sediment data...")

age_da = pygmt.grdsample('/Users/simon/Data/AgeGrids/2020/age.2020.1.GeeK2007.6m.nc',
                         region='g', spacing='5m')

sed_da = pygmt.grdsample('/Users/simon/GIT/pyBacktrack/pybacktrack/bundle_data/sediment_thickness/GlobSed.nc',
                         region='g', spacing='5m')

age_da = af.extend_longitude_range(age_da).sel(lon=slice(-190, 190))
sed_da = af.extend_longitude_range(sed_da).sel(lon=slice(-190, 190))

sed_da = sed_da.where(np.isfinite(sed_da), 1.)
sed_da = sed_da.where(sed_da < 1000., 1000.)

# Generate single random field for consistency across comparisons
rand_da = age_da.copy()
np.random.seed(42)  # For reproducibility
rand_da.data = af.generate_random_field(rand_da.data.shape)

# Select test region
xmin, xmax = -50, 20
ymin, ymax = -30, 0
age_da = age_da.sel(lon=slice(xmin, xmax), lat=slice(ymin, ymax))
sed_da = sed_da.sel(lon=slice(xmin, xmax), lat=slice(ymin, ymax))
rand_da = rand_da.sel(lon=slice(xmin, xmax), lat=slice(ymin, ymax))

print(f"\nRegion: {xmin}° to {xmax}° E, {ymin}° to {ymax}° N")
print(f"Grid shape: {age_da.shape}")
print(f"Age range: {np.nanmin(age_da.data):.1f} - {np.nanmax(age_da.data):.1f} Myr")
print(f"Sediment range: {np.nanmin(sed_da.data):.1f} - {np.nanmax(sed_da.data):.1f} m")

# Visualize input data
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
age_da.plot(ax=axes[0], cmap='viridis')
axes[0].set_title('Seafloor Age (Myr)', fontweight='bold')
sed_da.plot(ax=axes[1], vmin=0, vmax=1000, cmap='YlOrBr')
axes[1].set_title('Sediment Thickness (m)', fontweight='bold')
plt.tight_layout()
plt.show()

## Method 1: Original Method (Fixed Parameters)

In [ ]:
print("="*70)
print("METHOD 1: Original - Fixed Parameters + Gaussian Filter")
print("="*70)

params_fixed = {
    'H': 50,       # Base RMS height in meters
    'kn': 0.05,    # Characteristic wavenumber (normal to ridge) km⁻¹
    'ks': 0.2,     # Characteristic wavenumber (parallel to ridge) km⁻¹
    'D': 2.2       # Fractal dimension
}

print("\nParameters (fixed):")
for k, v in params_fixed.items():
    print(f"  {k}: {v}")
print(f"\nFilter type: Gaussian (default)")

In [ ]:
def process_bathymetry_chunk(coord, age_dataarray, sed_dataarray, rand_dataarray, 
                             chunksize, chunkpad, params, filter_type='gaussian'):
    """
    Process a single chunk of bathymetry.
    
    Updated to support filter_type parameter.
    """
    chunk_age = age_dataarray[coord[0]:coord[0]+chunksize+chunkpad, 
                               coord[1]:coord[1]+chunksize+chunkpad]
    chunk_sed = sed_dataarray[coord[0]:coord[0]+chunksize+chunkpad, 
                               coord[1]:coord[1]+chunksize+chunkpad]
    chunk_random = rand_dataarray[coord[0]:coord[0]+chunksize+chunkpad, 
                                   coord[1]:coord[1]+chunksize+chunkpad]
    
    if np.all(np.isnan(chunk_age.data)):
        return chunk_age
        
    # Generate the synthetic bathymetry with specified filter type
    synthetic_bathymetry = af.generate_bathymetry_spatial_filter(
        chunk_age.data, 
        chunk_sed.data / 5., 
        params,
        chunk_random.data,
        filter_type=filter_type
    )

    return xr.DataArray(
        synthetic_bathymetry, 
        coords=chunk_age.coords, 
        name='z'
    )[int(chunkpad/2):int(-chunkpad/2), int(chunkpad/2):int(-chunkpad/2)]


# Tiling parameters
full_ny, full_nx = age_da.shape
chunksize = 50
chunkpad = 20
chunkpad = int(2 * np.round(chunkpad / 2))  # Ensure even
num_cpus = 4

# Generate chunk coordinates
coords = np.meshgrid(np.arange(0, full_ny-1, chunksize), 
                     np.arange(0, full_nx-1, chunksize))
coords = list(zip(coords[0].flatten(), coords[1].flatten()))

print(f"\nProcessing {len(coords)} chunks (size={chunksize}, pad={chunkpad}, CPUs={num_cpus})")
print("This will take a few minutes...")

start = time.time()
results_method1 = Parallel(n_jobs=num_cpus)(delayed(process_bathymetry_chunk)(
    coord, age_da, sed_da, rand_da, chunksize, chunkpad, params_fixed, 'gaussian'
) for coord in coords)
elapsed = time.time() - start

results_method1 = [result for result in results_method1 if 0 not in result.shape]

print(f"\nCompleted in {elapsed:.1f} seconds ({len(results_method1)} valid chunks)")

## Method 2: NEW - Spreading Rate Derived Parameters

In [ ]:
print("="*70)
print("METHOD 2: NEW - Spreading Rate Derived Parameters + Gaussian Filter")
print("="*70)

# Calculate spreading rate from age gradient
# Grid spacing is 5 arcmin ~ 9.26 km at equator (use 9 km for this region)
grid_spacing_km = 9.0

print(f"\nCalculating spreading rate from age gradient...")
print(f"  Grid spacing: {grid_spacing_km} km")

spreading_rate = af.calculate_spreading_rate_from_age(age_da.data, grid_spacing_km=grid_spacing_km)

# Get median spreading rate (ignoring NaNs)
median_rate = np.nanmedian(spreading_rate)
mean_rate = np.nanmean(spreading_rate)

print(f"\nSpreading rate statistics:")
print(f"  Median: {median_rate:.1f} mm/yr")
print(f"  Mean: {mean_rate:.1f} mm/yr")
print(f"  Range: {np.nanpercentile(spreading_rate, 5):.1f} - {np.nanpercentile(spreading_rate, 95):.1f} mm/yr (5-95%)")

# Derive parameters from spreading rate
params_derived = af.spreading_rate_to_params(median_rate)

print(f"\nDerived parameters (from {median_rate:.1f} mm/yr):")
for k, v in params_derived.items():
    print(f"  {k}: {v:.3f}")
print(f"\nFilter type: Gaussian (default)")

# Visualize spreading rate
fig, ax = plt.subplots(figsize=(12, 6))
im = ax.imshow(spreading_rate, cmap='plasma', origin='lower', vmin=0, vmax=80)
ax.set_title(f'Calculated Spreading Rate (Median: {median_rate:.1f} mm/yr)', 
             fontweight='bold', fontsize=14)
ax.set_xlabel('X (grid cells)')
ax.set_ylabel('Y (grid cells)')
plt.colorbar(im, ax=ax, label='Half-spreading rate (mm/yr)')
plt.tight_layout()
plt.show()

In [ ]:
print(f"\nProcessing {len(coords)} chunks with derived parameters...")
print("This will take a few minutes...")

start = time.time()
results_method2 = Parallel(n_jobs=num_cpus)(delayed(process_bathymetry_chunk)(
    coord, age_da, sed_da, rand_da, chunksize, chunkpad, params_derived, 'gaussian'
) for coord in coords)
elapsed = time.time() - start

results_method2 = [result for result in results_method2 if 0 not in result.shape]

print(f"\nCompleted in {elapsed:.1f} seconds ({len(results_method2)} valid chunks)")

## Method 3: NEW - Von Kármán Filter

In [ ]:
print("="*70)
print("METHOD 3: NEW - Fixed Parameters + von Kármán Filter")
print("="*70)

print("\nUsing same fixed parameters as Method 1:")
for k, v in params_fixed.items():
    print(f"  {k}: {v}")
print(f"\nFilter type: von Kármán (Bessel function)")
print("  (Theoretically correct for fractal terrain)")

In [ ]:
print(f"\nProcessing {len(coords)} chunks with von Kármán filter...")
print("This will take a few minutes...")

start = time.time()
results_method3 = Parallel(n_jobs=num_cpus)(delayed(process_bathymetry_chunk)(
    coord, age_da, sed_da, rand_da, chunksize, chunkpad, params_fixed, 'von_karman'
) for coord in coords)
elapsed = time.time() - start

results_method3 = [result for result in results_method3 if 0 not in result.shape]

print(f"\nCompleted in {elapsed:.1f} seconds ({len(results_method3)} valid chunks)")

## Visualization: Side-by-Side Comparison

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(30, 24))

# Method 1: Original (Fixed params + Gaussian)
ax = axes[0]
for res in results_method1:
    ax.pcolormesh(res.lon, res.lat, res.data, vmin=-1, vmax=1, cmap='seismic')
ax.set_title('Method 1: Original - Fixed Parameters (H=50m) + Gaussian Filter', 
             fontweight='bold', fontsize=16)
ax.set_xlabel('Longitude (°E)', fontsize=12)
ax.set_ylabel('Latitude (°N)', fontsize=12)

# Method 2: Spreading rate derived + Gaussian
ax = axes[1]
for res in results_method2:
    ax.pcolormesh(res.lon, res.lat, res.data, vmin=-1, vmax=1, cmap='seismic')
ax.set_title(f'Method 2: NEW - Spreading Rate Derived (H={params_derived["H"]:.0f}m from {median_rate:.0f} mm/yr) + Gaussian Filter', 
             fontweight='bold', fontsize=16)
ax.set_xlabel('Longitude (°E)', fontsize=12)
ax.set_ylabel('Latitude (°N)', fontsize=12)

# Method 3: Fixed params + von Kármán
ax = axes[2]
for res in results_method3:
    ax.pcolormesh(res.lon, res.lat, res.data, vmin=-1, vmax=1, cmap='seismic')
ax.set_title('Method 3: NEW - Fixed Parameters (H=50m) + von Kármán Filter', 
             fontweight='bold', fontsize=16)
ax.set_xlabel('Longitude (°E)', fontsize=12)
ax.set_ylabel('Latitude (°N)', fontsize=12)

plt.tight_layout()
plt.savefig('tiling_comparison.png', dpi=150, bbox_inches='tight')
print("Saved: tiling_comparison.png")
plt.show()

## Statistics Summary

In [ ]:
# Calculate statistics for each method
def calc_stats(results):
    all_data = np.concatenate([res.data.flatten() for res in results])
    all_data = all_data[np.isfinite(all_data)]
    return {
        'mean': np.mean(all_data),
        'std': np.std(all_data),
        'min': np.min(all_data),
        'max': np.max(all_data),
        'p5': np.percentile(all_data, 5),
        'p95': np.percentile(all_data, 95)
    }

stats1 = calc_stats(results_method1)
stats2 = calc_stats(results_method2)
stats3 = calc_stats(results_method3)

print("="*70)
print("STATISTICS SUMMARY")
print("="*70)

print("\nMethod 1 (Original - Fixed H=50m + Gaussian):")
print(f"  RMS: {stats1['std']:.3f} m")
print(f"  Range: {stats1['min']:.3f} to {stats1['max']:.3f} m")
print(f"  5-95%: {stats1['p5']:.3f} to {stats1['p95']:.3f} m")

print(f"\nMethod 2 (NEW - Derived H={params_derived['H']:.0f}m + Gaussian):")
print(f"  RMS: {stats2['std']:.3f} m")
print(f"  Range: {stats2['min']:.3f} to {stats2['max']:.3f} m")
print(f"  5-95%: {stats2['p5']:.3f} to {stats2['p95']:.3f} m")
print(f"  RMS ratio vs Method 1: {stats2['std']/stats1['std']:.2f}×")

print(f"\nMethod 3 (NEW - Fixed H=50m + von Kármán):")
print(f"  RMS: {stats3['std']:.3f} m")
print(f"  Range: {stats3['min']:.3f} to {stats3['max']:.3f} m")
print(f"  5-95%: {stats3['p5']:.3f} to {stats3['p95']:.3f} m")
print(f"  RMS ratio vs Method 1: {stats3['std']/stats1['std']:.2f}×")

print("\n" + "="*70)
print("INTERPRETATION")
print("="*70)
print("\n• Method 1 (original): Baseline using fixed parameters")
print(f"• Method 2 (spreading rate): Automatically adjusts H based on local")
print(f"  spreading rate ({median_rate:.0f} mm/yr → H={params_derived['H']:.0f}m)")
print("• Method 3 (von Kármán): Uses theoretically correct filter with")
print("  heavier tails, producing slightly rougher texture")
print("\n• All methods produce realistic linear abyssal hill ridges")
print("• Choose method based on your needs:")
print("  - Method 1: Simple, fast, proven (recommended for most cases)")
print("  - Method 2: Data-driven parameter selection")
print("  - Method 3: Theoretically rigorous, slightly more detailed")
print("="*70)

## Zoomed Comparison

In [ ]:
# Find a representative chunk for detailed comparison
if len(results_method1) > 10:
    idx = len(results_method1) // 2  # Middle chunk
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    vmin, vmax = -1, 1
    
    im0 = axes[0].imshow(results_method1[idx].data, cmap='seismic', 
                         vmin=vmin, vmax=vmax, origin='lower')
    axes[0].set_title('Method 1: Fixed + Gaussian', fontweight='bold', fontsize=14)
    plt.colorbar(im0, ax=axes[0], label='Height (m)')
    
    im1 = axes[1].imshow(results_method2[idx].data, cmap='seismic', 
                         vmin=vmin, vmax=vmax, origin='lower')
    axes[1].set_title('Method 2: Derived + Gaussian', fontweight='bold', fontsize=14)
    plt.colorbar(im1, ax=axes[1], label='Height (m)')
    
    im2 = axes[2].imshow(results_method3[idx].data, cmap='seismic', 
                         vmin=vmin, vmax=vmax, origin='lower')
    axes[2].set_title('Method 3: Fixed + von Kármán', fontweight='bold', fontsize=14)
    plt.colorbar(im2, ax=axes[2], label='Height (m)')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nZoomed view of chunk {idx}")
    print(f"  Location: {results_method1[idx].lon.values[0]:.1f}°E, {results_method1[idx].lat.values[0]:.1f}°N")
    print(f"  Local RMS - Method 1: {np.std(results_method1[idx].data):.3f} m")
    print(f"  Local RMS - Method 2: {np.std(results_method2[idx].data):.3f} m")
    print(f"  Local RMS - Method 3: {np.std(results_method3[idx].data):.3f} m")

## Summary

This notebook demonstrated the new features in AbFab.py:

### New Features
1. **Dual filter support**: Choose between Gaussian (fast, simple) or von Kármán (theoretically correct)
2. **Spreading rate utilities**: Automatically derive optimal parameters from age gradient
3. **Backward compatible**: Original method still works exactly the same

### Recommendations
- **For production**: Use Method 1 (original) or Method 2 (spreading rate derived) with Gaussian filter
- **For research**: Try Method 3 (von Kármán) for theoretically rigorous results
- **Parameter selection**: Use spreading rate utilities when available data supports it

All methods produce realistic abyssal hill morphology with proper orientation!